In [1]:
# importeren van alle functies en het lezen van het databestand.
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import plotly.express as px
import cufflinks as cf
import numpy as np
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)
from sklearn.cluster import KMeans
df=pd.read_excel("DATA.xlsx")

In [2]:
# business/data understanding: hoe ziet de data er uit?
df
df.info();
df.describe().transpose();
df.columns;

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4684 entries, 0 to 4683
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Method name          4684 non-null   object 
 1   N (no. labs)         4684 non-null   object 
 2   Mean                 4602 non-null   float64
 3   Bias                 4058 non-null   float64
 4   CV                   4584 non-null   float64
 5   Sample               4684 non-null   object 
 6   Reference Value      4656 non-null   float64
 7   Year                 4684 non-null   int64  
 8   Source (CAP/EurA1c)  4684 non-null   object 
 9   Type (Fresh/frozen)  4684 non-null   object 
 10  Manufacturer         4684 non-null   object 
 11  Total Error          4684 non-null   float64
dtypes: float64(5), int64(1), object(6)
memory usage: 439.2+ KB


In [18]:
#data preparation: alleen de kolommen die van toepassing zijn, terug laten komen in de dataset.
df2 = df[["Mean","Bias","CV","Year","Manufacturer","Total Error"]];

In [4]:
df2

,Mean,Bias,CV,Year,Manufacturer,Total Error
0,7.96,-0.54,5.5,2011,Bayer/Metrika,10.240
1,5.14,-0.46,3.8,2012,Bayer/Metrika,6.988
2,6.37,-0.74,5.3,2013,Bayer,9.648
3,10.40,NaN,8.1,2006,Metrika,15.876
4,5.30,NaN,6.1,2007,Metrika,11.956
...,...,...,...,...,...,...
4679,8.22,NaN,2.6,2021,Vitros,5.096
4680,9.77,-0.28,2.9,2020,Vitros,5.404
4681,5.94,-0.04,2.1,2021,Vitros,4.076
4682,5.81,-0.18,2.5,2020,Vitros,4.720


In [5]:
df2 = df2.sort_values(["Manufacturer"])
df2["Manufacturer"].unique();
df2

,Mean,Bias,CV,Year,Manufacturer,Total Error
309,6.02,0.37,4.2,2014,Abbott,8.602
245,9.49,0.10,1.4,2021,Abbott,2.844
244,9.39,0.08,1.8,2019,Abbott,3.608
243,5.10,-0.03,1.1,2020,Abbott,2.126
242,5.51,0.20,4.1,2016,Abbott,8.236
...,...,...,...,...,...,...
4575,8.32,-0.07,2.3,2014,Vitros,4.438
4576,5.08,-0.22,3.3,2013,Vitros,6.248
4577,9.42,0.22,3.0,2011,Vitros,6.100
4579,5.81,-0.33,3.1,2013,Vitros,5.746


In [19]:
# data prep: Groeperen van manufacturer op jaar, en daarna de index setten op manufacturer
df3 = df2[df2["Manufacturer"]=='Abbott']

df4 = df2.groupby(["Manufacturer","Year"]).mean()
df4 = df4.reset_index()
df4 = df4.set_index('Manufacturer')
df4

,Year,Mean,Bias,CV,Total Error
Manufacturer,,,,,
Abbott,2006,8.155556,NaN,6.744444,13.219111
Abbott,2007,8.233333,NaN,4.500000,8.820000
Abbott,2008,8.400000,NaN,4.800000,9.408000
Abbott,2009,7.158333,0.183333,5.366667,10.610333
Abbott,2010,7.243333,0.026667,4.616667,9.075333
...,...,...,...,...,...
Vitros,2017,7.529333,0.074000,2.626667,5.222267
Vitros,2018,7.108000,-0.029333,2.573333,5.014400
Vitros,2019,7.080635,0.407647,3.064706,6.414471


In [7]:
# sns.pairplot(df4, hue='Manufacturer', kind='reg', corner=True, plot_kws={'line_kws':{'color':'red'}});

In [8]:
def plotPair(manufacturer):
    df3 = df2[df2["Manufacturer"]== manufacturer]
    sns.pairplot(df3, hue='Manufacturer', kind='reg', corner=True, plot_kws={'line_kws':{'color':'red'}});

In [9]:
interact(plotPair, manufacturer = df2['Manufacturer'].unique())

interactive(children=(Dropdown(description='manufacturer', options=('Abbott', 'Alere', 'Arkray', 'Axis-Shield'…

<function __main__.plotPair(manufacturer)>

In [10]:
df5 = df4.reset_index()

In [11]:
df5

,Manufacturer,Year,Mean,Bias,CV,Total Error
0,Abbott,2006,8.155556,NaN,6.744444,13.219111
1,Abbott,2007,8.233333,NaN,4.500000,8.820000
2,Abbott,2008,8.400000,NaN,4.800000,9.408000
3,Abbott,2009,7.158333,0.183333,5.366667,10.610333
4,Abbott,2010,7.243333,0.026667,4.616667,9.075333
...,...,...,...,...,...,...
208,Vitros,2017,7.529333,0.074000,2.626667,5.222267
209,Vitros,2018,7.108000,-0.029333,2.573333,5.014400
210,Vitros,2019,7.080635,0.407647,3.064706,6.414471
211,Vitros,2020,7.110667,-0.110667,2.693333,5.168267


In [12]:
# tel het aantal jaar dat een merk actief is. Mocht deze maar in 1 jaar voorkomen, haal die dan uit de dataset.
value_counts = df5["Manufacturer"].value_counts()
to_remove = value_counts[value_counts <= 1].index
ja = df5[~df5.Manufacturer.isin(to_remove)]
df5 = ja

In [13]:
lekkertellen = ja["Manufacturer"].value_counts()
lekkertellen

Abbott                       16
Beckman Coulter              16
Tosoh                        16
Roche Diagnostics            16
Bio-Rad                      16
Vitros                       16
Siemens                      14
Trinity Biotech              13
Sebia                         8
Roche Diagnostics/Hitachi     7
Axis-Shield                   7
Arkray                        6
Alere                         6
Olympus                       5
Not specified                 5
Menarini                      5
Metrika                       5
Primus                        4
Synchron                      4
Bayer                         3
Medinor                       3
HemoCue                       3
Bayer/Metrika                 3
JEOL                          2
EKF Diagnostics               2
Sysmex                        2
Thermo Scientific             2
Dade Behring                  2
BioMajesty                    2
Name: Manufacturer, dtype: int64

In [14]:
# Realiseren: laat de lijngrafieken zien die bij een specifieke manufacturer horen.
def lijngrafiek(manufacturer):
    df6 = df5[df5["Manufacturer"]== manufacturer]
    sns.relplot(data=df5, x="Year", y="CV", hue=df6.Manufacturer, kind="line")
    sns.relplot(data=df5, x="Year", y="Bias", hue=df6.Manufacturer, kind="line")
    sns.relplot(data=df5, x="Year", y="Total Error", hue=df6.Manufacturer, kind="line")

In [15]:
interact(lijngrafiek, manufacturer = df5['Manufacturer'].unique())

interactive(children=(Dropdown(description='manufacturer', options=('Abbott', 'Alere', 'Arkray', 'Axis-Shield'…

<function __main__.lijngrafiek(manufacturer)>

In [16]:
# regressievergelijking waarop kan worden voorspeld.
def regressie(manufacturer):
    df6 = df5[df5["Manufacturer"]== manufacturer]
    sns.regplot(data=df6, x="Year", y="CV")
    sns.regplot(data=df6, x="Year", y="Bias")
    sns.regplot(data=df6, x="Year", y="Total Error")

In [17]:
interact(regressie, manufacturer = df5['Manufacturer'].unique())

interactive(children=(Dropdown(description='manufacturer', options=('Abbott', 'Alere', 'Arkray', 'Axis-Shield'…

<function __main__.regressie(manufacturer)>